In [3]:
%%html
<p style = "font-size:300%; text-align:center; color:#336600"> <b> Naive Bayes</b></p>

# Definiție
---
Clasificatorii Naive Bayes sunt o colecție de algoritmi de clasificare bazați pe teorema Bayes. Toți algoritmii împart un principiu comun. <br>
**Teorma lui Bayes** este o teoremă fundamentală a teoriei probabilității, care determină probabilitatea apartenenței evenimentelor și a obiectelor la o anumită grupă.
# Problemă
---
Să se prezică dacă, în condițiile următoare, Safety Car-ul de la cursa de Formula 1 va ieși sau nu pe circuit: <br>
*   Condiții meteo: ploaie
*   Posibilitate depășire: bun
*   Tip circuit: normal
*   Supraîncălzire motor: nu <br>
Pentru ca Safety Car-ul să iasă pe circuit, ar trebui ca condițiile de cursă să poată provoca un incident, sau unul din monoposturi să aibă o problemă tehnică care să nu iî permită oprirea într-un loc sigur (linia de pit).

Importăm mai întâi librăriile necesare pentru rezolvarea problemei:
*   numpy pentru lucrul cu matrici
*   mathplotlib pentru desenarea graficului
*   pandas pentru a colecta datele intr-o structură de date, ușurând utilizarea lor de către algoritm
*   scikit-learn (sklearn) este cea mai utilă librărie folosită in machine learning în Python, conținând o mulțime de tool-uri eficiente pentru machine learning și modelare statistică.





In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn import preprocessing
from sklearn.naive_bayes import GaussianNB

Vom adăuga setul de date folosit pentru rezolvarea problemei noastre, după care vom afișa primele 5 poziții din tabel folosind head(). <br>
În setul nostru de date vom avea 5 informații diferite: condițiile meteo, posibilitatea de depășire a unui adversar, tipul circuitului, dacă există posibilitatea ca unei mașini să i se supraîncălzească motorul și prezența Safety Car-ului pe circuit. Cea din urmă va fi label-ul pe baza căruia se va face prezicerea.

In [5]:
race_data = {
    'weather_condition': ['rainy', 'rainy', 'sunny','cloudy', 'sunny', 'rainy','cloudy', 'rainy', 'sunny', 'sunny', 'rainy', 'cloudy', 'rainy', 'sunny'],
    'overtake_posibility': ['good', 'bad', 'bad','normal', 'good', 'normal','bad', 'good', 'bad', 'normal', 'normal', 'good', 'bad', 'good'],
    'track_type': ['normal', 'hard', 'normal','normal', 'hard', 'hard','normal', 'hard', 'normal', 'normal', 'normal', 'hard', 'normal', 'hard'],
    'engine_overheating': ['no', 'no', 'yes','no', 'yes', 'no','yes', 'yes', 'yes', 'no', 'no', 'no', 'no', 'yes'],
    'safety_car_on_track': ['no', 'yes', 'yes','no', 'yes', 'no','no', 'yes', 'yes', 'no', 'no', 'no', 'yes', 'yes']
}

df = pd.DataFrame(race_data, columns = ['weather_condition', 'overtake_posibility', 'track_type', 'engine_overheating', 'safety_car_on_track'])
df.head()


,weather_condition,overtake_posibility,track_type,engine_overheating,safety_car_on_track
0,rainy,good,normal,no,no
1,rainy,bad,hard,no,yes
2,sunny,bad,normal,yes,yes
3,cloudy,normal,normal,no,no
4,sunny,good,hard,yes,yes


Vom converti stringurile în numere folosind librăria LabelEncoder din Sckit-learn.

In [6]:
le = preprocessing.LabelEncoder()
weather_condition_encoded = le.fit_transform(df.weather_condition)
overtake_posibility_encoded = le.fit_transform(df.overtake_posibility)
track_type_encoded = le.fit_transform(df.track_type)
engine_overheating_encoded = le.fit_transform(df.engine_overheating)
label = le.fit_transform(df.safety_car_on_track)
print(weather_condition_encoded, overtake_posibility_encoded, track_type_encoded, engine_overheating_encoded, label, sep="\n")

[1 1 2 0 2 1 0 1 2 2 1 0 1 2]
[1 0 0 2 1 2 0 1 0 2 2 1 0 1]
[1 0 1 1 0 0 1 0 1 1 1 0 1 0]
[0 0 1 0 1 0 1 1 1 0 0 0 0 1]
[0 1 1 0 1 0 0 1 1 0 0 0 1 1]


Atfel:

*   "sunny" = 2, "rainy" = 1, iar "cloudy" = 0 pentru condițiile vremii
*   "normal" = 2, "good" = 1, iar "bad" = 0 pentru condițiile de depășire
*   "normal" = 1, iar "hard" = 0 pentru dificultatea circuitului
*   "yes" = 1, iar "no" = 0 pentru supraîncălizrea motorului și apariția Safety Car-ului pe circuit.







Acum vom crea liste de tupluri, pentru a crea o singură variabilă. Vom combina primele 4 feature-uri.

In [7]:
features = list(zip(weather_condition_encoded, overtake_posibility_encoded, track_type_encoded, engine_overheating_encoded))
print(features)

[(np.int64(1), np.int64(1), np.int64(1), np.int64(0)), (np.int64(1), np.int64(0), np.int64(0), np.int64(0)), (np.int64(2), np.int64(0), np.int64(1), np.int64(1)), (np.int64(0), np.int64(2), np.int64(1), np.int64(0)), (np.int64(2), np.int64(1), np.int64(0), np.int64(1)), (np.int64(1), np.int64(2), np.int64(0), np.int64(0)), (np.int64(0), np.int64(0), np.int64(1), np.int64(1)), (np.int64(1), np.int64(1), np.int64(0), np.int64(1)), (np.int64(2), np.int64(0), np.int64(1), np.int64(1)), (np.int64(2), np.int64(2), np.int64(1), np.int64(0)), (np.int64(1), np.int64(2), np.int64(1), np.int64(0)), (np.int64(0), np.int64(1), np.int64(0), np.int64(0)), (np.int64(1), np.int64(0), np.int64(1), np.int64(0)), (np.int64(2), np.int64(1), np.int64(0), np.int64(1))]


Acum vom crea un model folosind Naive Bayes. Vom crea clasificatorul, o să incărcăm datele în clasificator după care vom realiza predicția.

Importăm modelul Gaussian Naive Bayes, după care vom crea clasificatorul și vom antrena modelul folosind toate datele din tabel.

**GaussianNB** presupune că fiecare clasă urmează o distribuție Gaussiană.




In [8]:
model = GaussianNB()
model.fit(features, label)

GaussianNB()

Acum vom realiza predicția pentru inputul nostru, adică: ploaie, bun, normal, nu (rainy, good, normal, no = 0, 1, 1, 0)

In [9]:
predicted = model.predict([[1, 1, 1, 0]])
print(predicted)
if(predicted):
  print("Safety car will be on track")
else:
  print("Safety car will not be on track")

[0]
Safety car will not be on track


Se observă că pentru condițiile date mai sus, Safety Car-ul nu va ieși pe circuit, deoarece sunt condiții oarecum optime pentru ca o cursă să se desfășoare fără incidente.

Acum vom face un test pentru condiții rele de cursă: 1, 0, 0, 0 (rainy, bad, hard, no). Output-ul de această dată ar trebui să fie 1.

In [10]:
predicted = model.predict([[1, 0, 0, 0]])
print(predicted)
if(predicted):
  print("Safety car will be on track")
else:
  print("Safety car will not be on track")

[1]
Safety car will be on track


Se poate observa că pentru un circuit acoperit de apă, cu o dificultate mai mare, pe care nu prea se poate depăși, Safety Car-ul va ieși cel mai probabil pe circuit.

In [11]:
# prompt: evaluate the model metrics

from sklearn import metrics
# Split the data into training and testing sets (optional but good practice)
# from sklearn.model_selection import train_test_split
# X_train, X_test, y_train, y_test = train_test_split(features, label, test_size=0.2, random_state=42)

# # Train the model on the training data
# model.fit(X_train, y_train)

# Make predictions on the test data
# y_pred = model.predict(X_test)

# Since the original code trains on the full dataset and then makes predictions on new data,
# we will evaluate the model on the data it was trained on for demonstration purposes.
# In a real scenario, you would evaluate on a separate test set.
y_pred = model.predict(features)

# Evaluate the model using common metrics
accuracy = metrics.accuracy_score(label, y_pred)
precision = metrics.precision_score(label, y_pred)
recall = metrics.recall_score(label, y_pred)
f1 = metrics.f1_score(label, y_pred)
confusion_matrix = metrics.confusion_matrix(label, y_pred)

print("Model Evaluation Metrics:")
print(f"Accuracy: {accuracy:.2f}")
print(f"Precision: {precision:.2f}")
print(f"Recall: {recall:.2f}")
print(f"F1 Score: {f1:.2f}")
print("\nConfusion Matrix:")
confusion_matrix

Model Evaluation Metrics:
Accuracy: 0.93
Precision: 1.00
Recall: 0.86
F1 Score: 0.92

Confusion Matrix:


array([[7, 0],
       [1, 6]])